# Installing object detection API

In [ ]:

!cd /home/barrier_reef
!rm -rf ./models/

!git clone --depth 1 https://github.com/tensorflow/models/
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

Cloning into 'models'...
remote: Enumerating objects: 3203, done.
remote: Counting objects: 100% (3203/3203), done.
remote: Compressing objects: 100% (2717/2717), done.
remote: Total 3203 (delta 847), reused 1354 (delta 442), pack-reused 0
Receiving objects: 100% (3203/3203), 33.41 MiB | 24.28 MiB/s, done.
Resolving deltas: 100% (847/847), done.


In [ ]:
%%writefile models/research/setup.py

import os
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
    'tf-models-official==2.7.0',
    'tensorflow_io'
]

setup(
    name='object_detection',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    include_package_data=True,
    packages=(
        [p for p in find_packages() if p.startswith('object_detection')] +
        find_packages(where=os.path.join('.', 'slim'))),
    package_dir={
        'datasets': os.path.join('slim', 'datasets'),
        'nets': os.path.join('slim', 'nets'),
        'preprocessing': os.path.join('slim', 'preprocessing'),
        'deployment': os.path.join('slim', 'deployment'),
        'scripts': os.path.join('slim', 'scripts'),
    },
    description='Tensorflow Object Detection Library',
    python_requires='>3.6',
)


Writing models/research/setup.py


In [ ]:
!python -m pip install models/research

Processing ./models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 23.1 MB 1.3 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 90 kB 7.3 MB/s 
     |████████████████████████████████| 596 kB 13.2 MB/s 
     |████████████████████████████████| 47.6 MB 62 kB/s 
     |████████████████████████████████| 1.1 MB 52.1 MB/s 
     |████████████████████████████████| 99 kB 7.1 MB/s 
     |████████████████████████████████| 213 kB 43.8 MB/s 
     |████████████████████████████████| 1.2 MB 16.5 MB/s 
   

# Unzipping data to machine to ensure faster training

In [ ]:
!unzip "/content/drive/MyDrive/deep learning/Object_detection/Coral_Reef/data/tensorflow-great-barrier-reef.zip" -d "/home/barrier_reef"

Streaming output truncated to the last 5000 lines.
  inflating: /home/barrier_reef/train_images/video_2/4303.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4304.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4305.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4306.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4307.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4308.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4309.jpg  
  inflating: /home/barrier_reef/train_images/video_2/431.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4310.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4311.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4312.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4313.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4314.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4315.jpg  
  inflating: /home/barrier_reef/train_images/video_2/4

#Importing libraries

In [ ]:
import pandas as pd
from pathlib import Path
from math import ceil
import json

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import zipfile
import io
import scipy.misc
import numpy as np

import glob
import imageio
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

# Auxilary functions

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

def parse_box_dict(box_dict):
  x_mins=[]
  y_mins=[]
  y_maxs=[]
  x_maxs=[]

  box_dict=json.loads(box_dict.replace("'", '"'))
  for v in (box_dict):
    x_mins.append(v['x'])
    x_maxs.append(v['x']+v['width'])
    y_mins.append(v['y'])
    y_maxs.append(v['y']+v['height'])

  return (y_mins, x_mins, y_maxs, x_maxs)

def draw_image_with_boxes(image_number,test=False):
  if (~test):
    vid_id,frame_num=train_df.loc[image_number,['video_id','video_frame']].values
  else:
    vid_id,frame_num=test_df.loc[image_number,['video_id','video_frame']].values

  path_to_image=path_to_data/'train_images'/f"video_{vid_id}"/f"{frame_num}.jpg"
  image = Image.open(path_to_image)

  box_dict=train_df.loc[image_number,'annotations']
  
  for y_min,x_min,ymax,x_max in zip(*parse_box_dict(box_dict)):
    viz_utils.draw_bounding_box_on_image(image,
                                         y_min,
                                         x_min,
                                         y_max,
                                         x_max,
                                         use_normalized_coordinates=False)
  
  return image

# Perform train/test split

As the images are part of the same video, it makes sense to split them, in a similiar way to Time Series split.

Alternatively can use sequences or random split

In [ ]:
path_to_data=Path('/home/barrier_reef')
data=pd.read_csv(path_to_data/'train.csv')

In [ ]:
#data=pd.read_csv(path_to_data/'train.csv')

#index_values=data.groupby('video_id').apply(lambda x: max(x.index)).rename('last_index').reset_index()
#index_values['first_index']=(index_values['last_index']*(1-test_share)+ \
#                             index_values['last_index'].shift(1).fillna(0)*test_share).apply(ceil)

#test_filter=np.zeros(data.shape[0])

#for _,val in index_values.iterrows():
#  test_filter[val['first_index']:val['last_index']+1]=1

#test_df=data[test_filter==1]
#train_df=data[test_filter!=1]

#train_df=(train_df[train_df['annotations']!='[]']).reset_index(drop=True)
#test_df=(test_df[test_df['annotations']!='[]']).reset_index(drop=True)

In [ ]:
data.groupby(['video_id'])['sequence_frame'].nunique()

video_id
0    1423
1    2959
2    2988
Name: sequence_frame, dtype: int64

In [ ]:
data.groupby(['video_id','sequence'])['sequence_frame'].nunique()

video_id  sequence
0         996          923
          8399        1423
          35305        853
          40258        480
          45015        617
          45518        798
          53708       1077
          59337        537
1         8503        2843
          15827        770
          17665         87
          18048         71
          29424        184
          44160        151
          60510       1167
          60754       2959
2         22643       1248
          26651       1525
          29859       2988
          37114       2800
Name: sequence_frame, dtype: int64

# Visualising the images

Showing functions that show a couple of examples of COTS

# Creation TF records from images

IMages have a shape of (720, 1280, 3)

In [ ]:
from object_detection.utils import dataset_util
from object_detection.dataset_tools import tf_record_creation_util
import contextlib2

In [ ]:
tfrecords_dir = Path("home/tfrecords")


if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)

In [ ]:
def image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )
    

In [ ]:
def create_example(row):

  box_dict=row['annotations']
  y_mins,x_mins, ymaxs, x_maxs=parse_box_dict(box_dict)

  image_path=path_to_data/'train_images'/f"video_{row['video_id']}"/f"{row['video_frame']}.jpg"
  im = tf.io.decode_jpeg(tf.io.read_file(str(image_path)))
  y_mins, x_mins, y_maxs, x_maxs=parse_box_dict(box_dict)
  n_classes=len(y_mins)
  classes=[('COTS').encode()]*n_classes
  labels=[1]*n_classes

  tf_example = tf.train.Example(features=tf.train.Features(feature={
      'height': dataset_util.int64_feature(im.shape[0]),
      'width': dataset_util.int64_feature(im.shape[1]),
      'detections_number':dataset_util.int64_feature(n_classes),
      'image': image_feature(im),
      'path': dataset_util.bytes_feature(str(image_path).encode()),
      'sequence_id':dataset_util.int64_feature(row['sequence']),
      'video_id':dataset_util.int64_feature(row['video_id']),
      'video_frame':dataset_util.int64_feature(row['video_frame']),
      'sequence_frame':dataset_util.int64_feature(row['sequence_frame']),
      'bbox/xmin': dataset_util.float_list_feature(x_mins),
      'bbox/xmax': dataset_util.float_list_feature(x_maxs),
      'bbox/ymin': dataset_util.float_list_feature(y_mins),
      'bbox/ymax': dataset_util.float_list_feature(y_maxs),
      'class/text': dataset_util.bytes_list_feature(classes),
      'class/label': dataset_util.int64_list_feature(labels)}))
  
  return tf_example

In [ ]:
def convert_to_tfrecord(data_df, tfrecords_dir, num_shards = 20):
  """Convert the object detection dataset to TFRecord as required by the TF ODT API."""

  if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)
    
  with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
        tf_record_close_stack, tfrecords_dir, num_shards)
    
    for index, row in data_df.iterrows():
      if index % 100 == 0:
        print('Processed {0} images.'.format(index))
      tf_example = create_example(row)
      output_shard_index = index % num_shards
      output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
  
  print('Completed processing {0} images.'.format(len(data_df)))

In [ ]:
tfrecords_dir = Path("/home/tfrecords/full_data/full_data")

convert_to_tfrecord(data, 
                    tfrecords_dir=tfrecords_dir, 
                    num_shards = 40)

tfrecords_dir = Path("/home/tfrecords/seen_data/seen_data")

anot_data=data[data.annotations.str.len()>2]
convert_to_tfrecord(anot_data, 
                    tfrecords_dir=tfrecords_dir, 
                    num_shards = 16)

Processed 0 images.
Processed 100 images.
Processed 200 images.
Processed 300 images.
Processed 400 images.
Processed 500 images.
Processed 600 images.
Processed 700 images.
Processed 800 images.
Processed 900 images.
Processed 1000 images.
Processed 1100 images.
Processed 1200 images.
Processed 1300 images.
Processed 1400 images.
Processed 1500 images.
Processed 1600 images.
Processed 1700 images.
Processed 1800 images.
Processed 1900 images.
Processed 2000 images.
Processed 2100 images.
Processed 2200 images.
Processed 2300 images.
Processed 2400 images.
Processed 2500 images.
Processed 2600 images.
Processed 2700 images.
Processed 2800 images.
Processed 2900 images.
Processed 3000 images.
Processed 3100 images.
Processed 3200 images.
Processed 3300 images.
Processed 3400 images.
Processed 3500 images.
Processed 3600 images.
Processed 3700 images.
Processed 3800 images.
Processed 3900 images.
Processed 4000 images.
Processed 4100 images.
Processed 4200 images.
Processed 4300 images.


In [ ]:
!cp -r "/home/tfrecords" "/content/drive/MyDrive/deep learning/Object_detection/Coral_Reef/tf_records_data"